**Índice**

- Leitura de arquivos (.csv)
- Validação: pandera (schema)
- Limpeza: Retirar/substituir valores ausentes (uso de pandas, numpy)
- Transformação:  
                 - Uso de filtros (.loc)
                 - Criação de novos dataframes específicos
                 - Pickle (para ML <-> label encoder)

# **Etapa de Extração**

In [ ]:
!pip install pandera

     |████████████████████████████████| 79 kB 6.5 MB/s 


In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';')
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,03/01/2010,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,03/01/2010,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,03/01/2010,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,04/01/2010,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,05/01/2010,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,29/12/2020,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,30/12/2020,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,30/12/2020,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,31/12/2020,09:00:00,0


In [ ]:
df.dtypes

codigo_ocorrencia            int64
codigo_ocorrencia2           int64
ocorrencia_classificacao    object
ocorrencia_cidade           object
ocorrencia_uf               object
ocorrencia_aerodromo        object
ocorrencia_dia              object
ocorrencia_hora             object
total_recomendacoes          int64
dtype: object

In [ ]:
#como dia é formato data, trocamos:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';', parse_dates=['ocorrencia_dia'])
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


In [ ]:
df.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0


In [ ]:
df.ocorrencia_dia.dt.month

0        3
1        3
2        3
3        4
4        5
        ..
5747    12
5748    12
5749    12
5750    12
5751    12
Name: ocorrencia_dia, Length: 5752, dtype: int64

# **Etapa de Validação**

**Validação 1: consertando a data (dia-mes trocado)**

In [ ]:
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';', parse_dates=['ocorrencia_dia'], dayfirst=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


**Essa etapa aqui serve para validar nosso dados, pois não basta só consertar a data da forma acima, pois existem casos que a data pode estar fora do padrão (exemplo colocar mês 15, sendo que pode até 12) e o código acima não faz a identificação disso.**

*biblioteca panderas para criar um schema.*

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

*se passar, é pq tá tudo ok; senão, volta pra célula anterior e modifica algum parâmetro
i.e: nullable=True* *texto em itálico*

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Fazendo uma validação do horário, coluna 'ocorrencia_hora', usando expressão regulares, limitando os valores para até 23:59:59*

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Fazendo uma validação do horário, coluna 'ocorrencia_uf', usando expressão regulares, limitando a string para no min 2 e max 2*

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


*Caso estejamos usando uma alteração de colunas, onde as vezes ela é requerida,
as vezes não, usamos o required.*

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'coluna_variavel': pa.Column(pa.String, required=False),
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

# **Etapa de Limpeza**

*Iremos usar uma função para certificar se realmente uma determinada coluna contém valores únicos em comparação as outras linhas*

In [ ]:
df.codigo_ocorrencia.is_unique

True

*Como essa coluna contém valores únicos, iremos colocar o índice de cada linha desse df para os valores contidos nessa coluna*
*`não é necessário, porém ajuda em diminuir futuros erros`*

In [ ]:
df.set_index('codigo_ocorrencia', inplace= True) #inplace para atualizar diretamente o df
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


In [ ]:
df.reset_index(drop=True, inplace=True)
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,2,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29,19:00:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30,18:30:00,0
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30,00:54:00,0
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31,09:00:00,0


*Realizando backup em uma coluna*

In [ ]:
df['ocorrencia_aerodromo_backup'] = df.ocorrencia_aerodromo
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_aerodromo_backup
codigo_ocorrencia,,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0,****
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,SBBE
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,SBRJ
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0,****
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,SBPK
...,...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0,SBKP
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0,****
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0,SBGL


*Excluindo uma coluna toda*

In [ ]:
df.drop('ocorrencia_aerodromo_backup', axis = 1, inplace = True)
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


*Modificando uma coluna toda*

In [ ]:
df['ocorrencia_aerodromo'] = 2
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
codigo_ocorrencia,,,,,,,,
40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,12:00:00,0
40349,40349,INCIDENTE,BELÉM,PA,2,2010-01-03,11:05:00,0
40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03,03:00:00,0
39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04,17:30:00,0
40324,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
79804,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29,19:00:00,0
79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30,18:30:00,0
79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30,00:54:00,0


*Modificando uma linha toda*

In [ ]:
df.loc[1] = 'oi'
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,1
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,12:00:00,0,oi
1,oi,oi,oi,oi,oi,oi,oi,oi,oi
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,03:00:00,0,oi
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04 00:00:00,17:30:00,0,oi
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05 00:00:00,19:25:00,0,oi
...,...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29 00:00:00,19:00:00,0,oi
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30 00:00:00,18:30:00,0,oi
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30 00:00:00,00:54:00,0,oi
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31 00:00:00,09:00:00,0,oi


*Modificando um valor específico no dataframe*

In [ ]:
df.loc[1, 'codigo_ocorrencia2'] = 'ola'
df

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,1
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,12:00:00,0,oi
1,ola,oi,oi,oi,oi,oi,oi,oi,oi
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,2,2010-01-03 00:00:00,03:00:00,0,oi
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,2,2010-01-04 00:00:00,17:30:00,0,oi
4,40324,INCIDENTE,PELOTAS,RS,2,2010-01-05 00:00:00,19:25:00,0,oi
...,...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,2,2020-12-29 00:00:00,19:00:00,0,oi
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,2,2020-12-30 00:00:00,18:30:00,0,oi
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,2,2020-12-30 00:00:00,00:54:00,0,oi
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,2,2020-12-31 00:00:00,09:00:00,0,oi


**fiz um reset (compilei o df novamente) a partir daqui**

Fazendo uma varredura dos valores incorretos no .csv, temos:

ocorrencia_uf
**

ocorrencia_aerodromo
###
####
\***
\****

ocorrencia_hora
NULL


In [ ]:
df.replace(['**', '###!', '####', '****', '*****', 'NULL'], pd.NA, inplace=True)
df
#pd.NA para substituir valores não corretos no df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


In [ ]:
df.tail()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0
5751,79844,79844,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2020-12-31,13:24:00,0


In [ ]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

*Substituindo todos os valores colocados como NA/NULL pela função fillna*

In [ ]:
df.fillna('valoresCertos', inplace = True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,valoresCertos,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,valoresCertos,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,valoresCertos,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,valoresCertos,2020-12-31,09:00:00,0


*Caso queira fazer um fillna para uma coluna em específico:*

In [ ]:
df.replace(['valoresCertos'], pd.NA, inplace = True)

In [ ]:
df.isna().sum() #ou isnull.sum tanto faz

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.fillna(value={'ocorrencia_uf': 10}, inplace=True)

In [ ]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

*Retirar todas as linhas que contém um valor NA/NULL*

In [ ]:
df.dropna(inplace=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
...,...,...,...,...,...,...,...,...,...
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


Retirar uma linha específica que contém um valor NA/NULL

In [ ]:
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


In [ ]:
df.dropna(subset = ['ocorrencia_aerodromo', 'ocorrencia_hora'])

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
...,...,...,...,...,...,...,...,...,...
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


*Excluindo uma coluna inteira*

*Fazendo um backup de uma coluna primeiramente*

In [ ]:
df['ocorrencia_cidade_backup'] = df.ocorrencia_cidade
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_cidade_backup
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0,RIO DE JANEIRO
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,BELÉM
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,RIO DE JANEIRO
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0,LUCAS DO RIO VERDE
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,PELOTAS


In [ ]:
df.drop(labels=['ocorrencia_cidade_backup'], axis = 1, inplace = True)

In [ ]:
df.tail()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0
5751,79844,79844,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2020-12-31,13:24:00,0


In [ ]:
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


*Retirar linhas duplicadas*

In [ ]:
df.drop_duplicates(inplace = True)

# **Etapa de Transformação dos Dados**

In [ ]:
!pip install pandera

     |████████████████████████████████| 79 kB 3.1 MB/s 


*No caso iremos ler logo os valores com uma certa rapidez na limpeza, passando os missing values logo na leitura, usando o parâmetro na_values*

In [ ]:
import pandas as pd
import pandera as pa
valores_naoInformados = ['**', '###!', '####', '****', '*****', 'NULL']
df = pd.read_csv('/content/ocorrencia_2010_2020.csv', sep = ';', parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_naoInformados)

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9]):([0-5]?[0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


*count() indica o número de linhas que contém valores **diferentes** de NULL/NA/NaN*

In [ ]:
df.count()

codigo_ocorrencia           5752
codigo_ocorrencia2          5752
ocorrencia_classificacao    5752
ocorrencia_cidade           5752
ocorrencia_uf               5751
ocorrencia_aerodromo        3403
ocorrencia_dia              5752
ocorrencia_hora             5751
total_recomendacoes         5752
dtype: int64

*Localizar dados nulos/não informados*

In [ ]:
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [ ]:
df.loc[df.ocorrencia_uf.isnull()]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0


In [ ]:
filtro_ocorrencia_uf = df.ocorrencia_uf.isnull()
df.loc[filtro_ocorrencia_uf]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0


In [ ]:
filtro_ocorrencia_aerodromo = df.ocorrencia_aerodromo.isnull()
df[filtro_ocorrencia_aerodromo].head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
10,39789,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-01-10,21:30:00,0
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
17,39810,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0


*Filtrando coisas específicas*

In [ ]:
filtro_ocorrencia_classificacao = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro_ocorrencia_classificacao, ['ocorrencia_classificacao','ocorrencia_cidade', 'ocorrencia_dia', 'ocorrencia_hora']]

,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_dia,ocorrencia_hora
7,INCIDENTE GRAVE,CANUTAMA,2010-01-09,12:30:00
8,INCIDENTE GRAVE,CASCAVEL,2010-01-10,23:15:00
9,INCIDENTE GRAVE,PARÁ DE MINAS,2010-01-10,20:00:00
29,INCIDENTE GRAVE,CURITIBA,2010-01-23,16:36:00
35,INCIDENTE GRAVE,PALMAS,2010-01-26,16:50:00
...,...,...,...,...
5741,INCIDENTE GRAVE,NOVA MARINGÁ,2020-12-28,13:00:00
5744,INCIDENTE GRAVE,GOIATUBA,2020-12-29,12:00:00
5746,INCIDENTE GRAVE,MANOEL URBANO,2020-12-29,18:30:00
5748,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,2020-12-30,18:30:00


In [ ]:
filtro_ocorrencia_classificacao.sum()

737

(acima) 737 valores que são do tipo 'INCIDENTE GRAVE'

Combinando dois filtros

In [ ]:
filtro_total_recomendacoes = df.total_recomendacoes > 2
filtro_ocorrencia_classificacao = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'

df.loc[filtro_total_recomendacoes & filtro_ocorrencia_classificacao, ['ocorrencia_aerodromo', 'ocorrencia_classificacao', 'ocorrencia_dia', 'total_recomendacoes']].head(10)

,ocorrencia_aerodromo,ocorrencia_classificacao,ocorrencia_dia,total_recomendacoes
7,NaN,INCIDENTE GRAVE,2010-01-09,3
75,NaN,INCIDENTE GRAVE,2010-02-26,3
83,SBRB,INCIDENTE GRAVE,2010-03-10,3
177,NaN,INCIDENTE GRAVE,2010-05-08,5
198,SBBR,INCIDENTE GRAVE,2010-05-20,8
271,SSZW,INCIDENTE GRAVE,2010-07-10,5
552,SBRP,INCIDENTE GRAVE,2011-01-01,10
673,NaN,INCIDENTE GRAVE,2011-03-06,3
834,NaN,INCIDENTE GRAVE,2011-06-17,10
841,SSNH,INCIDENTE GRAVE,2011-06-19,5


In [ ]:
num_filtro = filtro_total_recomendacoes & filtro_ocorrencia_classificacao
num_filtro.sum()

46

*Filtro mais elaborado*

*usando método isin == operador "ou"*

In [ ]:
filtro_elaborado = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'ACIDENTE'])
filtro_elaborado2 = df.ocorrencia_uf.isin(['RJ', 'SP'])

df.loc[filtro_elaborado & filtro_elaborado2].head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
31,39323,39323,ACIDENTE,CASIMIRO DE ABREU,RJ,NaN,2010-01-24,16:15:00,0
34,39322,39322,ACIDENTE,SÃO BERNARDO DO CAMPO,SP,NaN,2010-01-26,13:50:00,2
37,39319,39319,ACIDENTE,IPERÓ,SP,NaN,2010-01-27,11:25:00,3
49,39295,39295,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2010-02-06,21:09:00,0
58,39275,39275,ACIDENTE,SÃO PAULO,SP,NaN,2010-02-10,09:03:00,3
61,41377,41377,ACIDENTE,SERRA NEGRA,SP,NaN,2010-02-14,17:30:00,0
63,39992,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
149,40447,40447,ACIDENTE,MANGARATIBA,RJ,NaN,2010-04-21,18:55:00,0


*Filtrando datas específicas*

In [ ]:
filtro_ano = df.ocorrencia_dia.dt.year == 2015
filtro_mes = df.ocorrencia_dia.dt.month == 12
filtro_dia = df.ocorrencia_dia.dt.day == 2

df.loc[filtro_ano & filtro_mes & filtro_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3431,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0


*Construindo uma nova coluna*

*data + horário*

*ocorrencia_dia + ocorrencia_hora*



In [ ]:
transformar = df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora #conversao do tipo da coluna data para string
df['ocorrencia_dia_hora'] = pd.to_datetime(transformar) #conversao da concatenacao acima para data novamente
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00


*Filtrando com base na data e hora*

In [ ]:
filtro_dia_hora1 = df.ocorrencia_dia_hora >= '2010-11-03 12:00:00'
filtro_dia_hora2 = df.ocorrencia_dia_hora <= '2010-12-03 12:00:00'

df.loc[filtro_dia_hora1 & filtro_dia_hora2, ['ocorrencia_uf','ocorrencia_dia_hora']].head(7)

,ocorrencia_uf,ocorrencia_dia_hora
469,SP,2010-11-04 10:55:00
470,ES,2010-11-05 09:06:00
471,PA,2010-11-05 13:00:00
472,SP,2010-11-05 22:36:00
473,MA,2010-11-07 19:30:00
474,SP,2010-11-08 14:30:00
475,RJ,2010-11-08 18:05:00
